In [1]:
# Financial Portfolio App with Account Balance Tracking

# Step 1: Install Necessary Libraries
# Uncomment the line below to install required packages if they are not already installed
# !pip install yfinance pandas matplotlib numpy

# Step 2: Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Configure matplotlib for inline plotting
%matplotlib inline

# Step 3: Define Functions to Fetch Stock Data

def get_stock_data(ticker, start_date, end_date):
    """
    Fetch historical data for a given stock ticker.
    """
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data[['Close']]

# Step 4: Define Account Class

class Account:
    def __init__(self, account_name, initial_balance=0.0):
        """
        Initialize an account with a name and balance.
        """
        self.account_name = account_name
        self.balance = initial_balance

    def deposit(self, amount):
        """
        Deposit funds into the account.
        """
        self.balance += amount

    def withdraw(self, amount):
        """
        Withdraw funds from the account, checking for sufficient funds.
        """
        if amount > self.balance:
            print(f"Insufficient funds in {self.account_name}. Transaction aborted.")
            return False
        else:
            self.balance -= amount
            return True

    def get_balance(self):
        """
        Return the current balance.
        """
        return self.balance

# Step 5: Define Portfolio Class with Enhanced Transaction Management

class Portfolio:
    def __init__(self):
        """
        Initialize an empty portfolio and a transaction history.
        """
        self.holdings = {}
        self.accounts = {}
        self.transaction_history = []

    def add_account(self, account_name, initial_balance=0.0):
        """
        Add a new account with an initial balance.
        """
        account = Account(account_name, initial_balance)
        self.accounts[account_name] = account

    def add_asset(self, ticker, quantity, purchase_price, account_name):
        """
        Add an asset to the portfolio, deducting the cost from the specified account.
        """
        if account_name not in self.accounts:
            print(f"Account {account_name} does not exist.")
            return
        
        total_cost = quantity * purchase_price
        if self.accounts[account_name].withdraw(total_cost):
            if ticker in self.holdings:
                self.holdings[ticker]['quantity'] += quantity
            else:
                self.holdings[ticker] = {'quantity': quantity, 'purchase_price': purchase_price}
            
            # Log transaction
            transaction = {
                'date': datetime.now(),
                'type': 'buy',
                'ticker': ticker,
                'quantity': quantity,
                'price': purchase_price,
                'account': account_name
            }
            self.transaction_history.append(transaction)
            print(f"Purchased {quantity} shares of {ticker} at {purchase_price} using {account_name}.")
        else:
            print("Transaction failed due to insufficient funds.")

    def sell_asset(self, ticker, quantity, sale_price, account_name):
        """
        Sell an asset, adding the proceeds to the specified account.
        """
        if ticker not in self.holdings or self.holdings[ticker]['quantity'] < quantity:
            print(f"Not enough {ticker} shares to sell.")
            return
        
        # Update holdings
        self.holdings[ticker]['quantity'] -= quantity
        if self.holdings[ticker]['quantity'] == 0:
            del self.holdings[ticker]

        # Calculate sale proceeds and deposit into the account
        proceeds = quantity * sale_price
        self.accounts[account_name].deposit(proceeds)
        
        # Log transaction
        transaction = {
            'date': datetime.now(),
            'type': 'sell',
            'ticker': ticker,
            'quantity': quantity,
            'price': sale_price,
            'account': account_name
        }
        self.transaction_history.append(transaction)
        print(f"Sold {quantity} shares of {ticker} at {sale_price} using {account_name}.")

    def portfolio_value(self, date="2023-01-01"):
        """
        Calculate the current value of the portfolio based on latest price.
        """
        total_value = 0
        for ticker, info in self.holdings.items():
            quantity = info['quantity']
            latest_price = get_stock_data(ticker, date, date)['Close'].iloc[0]
            total_value += quantity * latest_price
        return total_value

    def account_summary(self):
        """
        Print a summary of all accounts and balances.
        """
        print("Account Summary:")
        for account_name, account in self.accounts.items():
            print(f"{account_name}: ${account.get_balance():.2f}")

    def transaction_summary(self):
        """
        Print a summary of all transactions.
        """
        print("Transaction History:")
        for transaction in self.transaction_history:
            print(transaction)

# Step 6: Example Usage

# Initialize portfolio
portfolio = Portfolio()

# Add accounts to the portfolio
portfolio.add_account("Bank Account", 5000)
portfolio.add_account("Stock Account", 10000)

# Display account summary
portfolio.account_summary()

# Add assets with account deduction
portfolio.add_asset("AAPL", 10, 150, "Bank Account")  # Buy AAPL shares
portfolio.add_asset("MSFT", 5, 250, "Stock Account")  # Buy MSFT shares

# Sell assets with account addition
portfolio.sell_asset("AAPL", 5, 160, "Bank Account")  # Sell AAPL shares

# Display updated account summary and portfolio value
portfolio.account_summary()
print("Portfolio Value:", portfolio.portfolio_value())

# Display transaction history
portfolio.transaction_summary()


ModuleNotFoundError: No module named 'yfinance'